In [16]:
import pandas as pd 
import numpy as np
import string, re
import nltk
from sklearn.cross_validation import train_test_split
from sklearn.feature_extraction.text import HashingVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer

from sklearn.pipeline import make_pipeline
from sklearn.cluster import KMeans
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from xgboost import XGBClassifier

from time import time

%matplotlib inline

In [4]:
data = pd.read_csv('Data/tweets.txt', sep = ';~;', engine='python')
PositiveTweets = pd.read_csv('Data/tweetsPositive.txt', sep = ';~;', engine='python')
NegativeTweets = pd.read_csv('Data/tweetsNegative.txt', sep = ';~;', engine='python')

emoji_list = pd.read_csv('Data/emoji_table.txt', encoding='utf-8', index_col=0).index.values
SentimentEmoji = pd.read_csv('Data/Emoji_classification.csv', encoding='utf-8').dropna()
SentimentHashtags = pd.read_csv('Data/hashtags.csv', encoding='utf-8').dropna()

## The test set for hillary
hillaryTest = pd.read_csv('Hillary.csv')
TrumpTest = pd.read_csv('Trump.csv')

In [5]:
# List of positive and negative tweets
sad = [':‑(', ':(', ':‑c', ':c', ':‑<', ':<', ':‑[' ,':[', ':-||', '>:[', ':{', ':@', '>:(']
Positive = [':‑)',':)', ':-]', ':]',':-3', ':3', ':->', ':>' ,'8-)', '8)',':-}', ':}', ':o)', ':c)', ':^)' ,'=]', '=)'
           ,':‑D', ':D', '8‑D', '8D', 'x‑D', 'xD', 'X‑D', 'XD', '=D', '=3', 'B^D']
SentimentHashtags['HashtagSentiment'] = SentimentHashtags['HashtagSentiment'].map({'Positive':1, 'Negative':-1})
SentimentEmoji['Sentiment'] = SentimentEmoji['Sentiment'].map({'Positive':1, 'Negative':-1, 'Neutral':0}).dropna()
SentimentHashtags['Directed'] = SentimentHashtags['Directed'].map({'T':1, 'H':0})
hillaryTest.Sentiment = hillaryTest.Sentiment.map({'Positive':1, 'Negative':-1, 'Neutral':0})
TrumpTest.Sentiment = TrumpTest.Sentiment.map({'Positive':1, 'Negative':-1, 'Neutral':0})

In [6]:
stop_list = nltk.corpus.stopwords.words('english') + ["rt"] # rt - stands for retweet
lemmatizer = nltk.stem.WordNetLemmatizer()

# regex for capturing tweets
reg = '(\:\w+\:|\<[\/\\]?3|[\(\)\\\D|\*\$][\-\^]?[\:\;\=]|[\:\;\=B8][\-\^]?[3DOPp\@\$\*\\\)\(\/\|])(?=\s|[\!\.\?]|$)'
emoticons = "|".join(map(re.escape, sad + Positive))

emoji_pattern = re.compile(u'('
    u'\ud83c[\udf00-\udfff]|'
    u'\ud83d[\udc00-\ude4f\ude80-\udeff]|'
    u'[\u2600-\u26FF\u2700-\u27BF])+', 
    re.UNICODE)
classifier =[]
def preprocess(tweet):
    # only processing if the the value is a string
    if type(tweet)!=type(2.0):
        tweet = tweet.decode('latin-1').encode("utf-8").decode('utf-8').strip()
        tweet = tweet.lower()
        # Removing hashtags
        tweet = " ".join(tweet.split('#'))
        # Removing URLs
        tweet = re.sub('((www\.[^\s]+)|(https://[^\s]+))','',tweet)
        tweet = re.sub('((pic\.[^\s]+)|(https://[^\s]+))','',tweet)
        tweet = re.sub("(http\S+)|(https\S+)", '', tweet)
        # Adding this pattern to the last cause it will remove everything after the start of a URL
        tweet = re.sub(u'[a-zA-Z0-9./]+\.[a-zA-Z0-9./ ]+.*$','',tweet)
        
        # Removing User mentions
        tweet = re.sub('@[^\s]+','',tweet)
        tweet = tweet.strip('\'"')
        # Removing stop words - This can be moved to count vectorization
        # tweet  = " ".join([word for word in tweet.split(" ") if word not in stop_list])
        # lemmatizing words 
        tweet = " ".join([lemmatizer.lemmatize(word) for word in tweet.split(" ")])
    else:
        tweet=''
    return tweet

def extractEmoticons(tweet):
    # emoji = emoji_pattern.findall(tweet)
    emoji = []
    for emo in emoji_list:
        if emo in tweet:
            emoji.append(emo)
    
    # these are :) :-) and other stuff
    emoticons = re.findall(reg, tweet)
    return " , ".join(emoji + emoticons)
def removeEmoticons(tweet):
    return re.sub(reg,'',tweet)

#Processing the tweets
data['processed_text'] = data.text.apply(preprocess)
hillaryTest['processed_text'] = hillaryTest.processed_text.apply(preprocess)
TrumpTest['processed_text'] = TrumpTest.processed_text.apply(preprocess)
PositiveTweets['processed_text'] = PositiveTweets.text.apply(preprocess)
NegativeTweets['processed_text'] = NegativeTweets.text.apply(preprocess)


#getting the emoticons from the cleaned data
data['emoticons'] = data['processed_text'].apply(extractEmoticons)

# Removing emoticons from the text data
data['processed_text'] = data['processed_text'].apply(removeEmoticons)

data = data.append(PositiveTweets).append(NegativeTweets)
print 'Completed'

Completed


In [9]:
HillaryTweets = data[data['processed_text'].str.contains('((hil.?ary)|(clinton))', case = False)]
DonaldTweets = data[data['processed_text'].str.contains('trump', case = False)]

datasets = [HillaryTweets.copy(), DonaldTweets.copy()]
TrainSets = []
for i, dataset in enumerate(datasets):
    hashtags = datasets[i]['hashtags'].copy().str.split(' ').apply(pd.Series, 1).stack()
    hashtags.index = hashtags.index.droplevel(-1)
    datasets[i].drop('hashtags', axis=1, inplace=True)
    hashtags.name = 'hashtags'
    
    datasets[i] = datasets[i].join(hashtags.str.strip())
    
    emoticons = datasets[i]['emoticons'].copy().str.split(' ').apply(pd.Series, 1).stack()
    emoticons.index = emoticons.index.droplevel(-1)
    datasets[i].drop('emoticons', axis=1, inplace=True)
    emoticons.name = 'emoticons'
    datasets[i] = datasets[i].join(emoticons.str.strip())
    
    Directed_hashtags = SentimentHashtags[SentimentHashtags['Directed'] == 0]
    Opp_hashtags = SentimentHashtags[SentimentHashtags['Directed'] != 0]
    Opp_hashtags.loc[: ,'HashtagSentiment'] = Opp_hashtags.HashtagSentiment * -1;
    
    Directed_hashtags = Directed_hashtags.append(Opp_hashtags)
    datasets[i] = pd.merge(datasets[i], Directed_hashtags, on = 'hashtags', how='outer')
    datasets[i] = pd.merge(datasets[i], SentimentEmoji, on = 'emoticons', how='outer')
    datasets[i]['Sentiment'] = datasets[i]['HashtagSentiment'].add(datasets[i]['Sentiment'], fill_value = 0)
    TrainSets.append(datasets[i][['username', 'date', 'processed_text', 'Sentiment']].dropna().groupby(['processed_text', 'Sentiment']).max().reset_index())

data_train = TrainSets[0][['processed_text','Sentiment']].copy().dropna()

/Users/shashankbhushan/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:1: UserWarning: This pattern has match groups. To actually get the groups, use str.extract.
  if __name__ == '__main__':
/Users/shashankbhushan/anaconda2/lib/python2.7/site-packages/pandas/core/indexing.py:476: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


In [22]:
vectorizers = [CountVectorizer(stop_words=stop_list), HashingVectorizer(stop_words=stop_list), TfidfVectorizer(stop_words=stop_list)]
vectorizersName = ['Count', 'Hash', 'TF-IDF']
for k, vectorizer in enumerate(vectorizers):
    X = vectorizer.fit_transform(data_train.processed_text.append(TrumpTest.processed_text))
    X_train = X[0:data_train.processed_text.shape[0]]
    Y_train = data_train['Sentiment']
    X_test = X[data_train.processed_text.shape[0]:]
    models = [RandomForestClassifier(), GradientBoostingClassifier(), XGBClassifier()]
    modelsName = ['RandomForest', 'Gradient Boosting', 'XGB Classifier']
    for m, model in enumerate(models):
        model.fit(X_train, Y_train)
        preds = model.predict(X_test.todense())
        score = 0
        tot = 0
        for i, pred in enumerate(preds):
            if(hillaryTest.Sentiment[i] == hillaryTest.Sentiment[i]):
                tot+=1
                if(hillaryTest.Sentiment[i] == pred):
                    score+=1

        print 'The score for Vectorizer:', vectorizersName[k],', Model:', modelsName[m], score, tot

0 0
The score for Vectorizer: Count , Model: RandomForest 45 72
0 1
The score for Vectorizer: Count , Model: Gradient Boosting 44 72
0 2
The score for Vectorizer: Count , Model: XGB Classifier 22 72
1 0
The score for Vectorizer: Hash , Model: RandomForest 45 72
1 1
The score for Vectorizer: Hash , Model: Gradient Boosting 45 72
1 2
The score for Vectorizer: Hash , Model: XGB Classifier 22 72
2 0
The score for Vectorizer: TF-IDF , Model: RandomForest 45 72
2 1
The score for Vectorizer: TF-IDF , Model: Gradient Boosting 44 72
2 2
The score for Vectorizer: TF-IDF , Model: XGB Classifier 22 72


In [47]:
vectorizer = TfidfVectorizer(stop_words=stop_list)
X = vectorizer.fit_transform(data_train.processed_text.append(TrumpTest.processed_text))
X_train = X[0:data_train.processed_text.shape[0]]
Y_train = data_train['Sentiment']
X_test = X[data_train.processed_text.shape[0]:]
models = [RandomForestClassifier(n_estimators = 10)]
for m, model in enumerate(models):
        model.fit(X_train, Y_train)
        preds = model.predict(X_test.todense())
        score = 0
        tot = 0
        for i, pred in enumerate(preds):
            if(TrumpTest.Sentiment[i] == TrumpTest.Sentiment[i]):
                tot+=1
                if(TrumpTest.Sentiment[i] == pred):
                    score+=1

        print 'The score for Vectorizer:', vectorizersName[k],', Model: Random Forest', score, tot

The score for Vectorizer: TF-IDF , Model: Random Forest 31 84


In [69]:
import logging
from gensim.models import word2vec

def get_words(tweet):
    return tweet.split(' ')
tweets = pd.Series(data['processed_text'].unique()).apply(get_words)

logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s',\
    level=logging.INFO)

# Set values for various parameters
num_features = 140    # Word vector dimensionality                      
min_word_count = 10   # Minimum word count                        
num_workers = 4       # Number of threads to run in parallel
context = 10          # Context window size                                                                                    
downsampling = 1e-3   # Downsample setting for frequent words

# Initialize and train the model (this will take some time)

print "Training model..."
model = word2vec.Word2Vec(tweets, workers=num_workers, size=num_features, min_count = min_word_count, window = context,
                          sample = downsampling)

# If you don't plan to train the model any further, calling 
# init_sims will make the model much more memory-efficient.
model.init_sims(replace=True)

# It can be helpful to create a meaningful model name and 
# save the model for later use. You can load it later using Word2Vec.load()
model_name = "30features_40minwords_10context"
model.save(model_name)

Training model...


In [8]:
len(data)

135000

In [44]:
data_train = TrainSets[1][['processed_text','Sentiment']].copy().dropna()

In [43]:
TrumpTest['processed_text'] = TrumpTest.processed_text.apply(preprocess)